In [1]:
import cv2
import numpy as np

In [2]:
# 定义一个自己读取图像的函数
def myimshow(arr):
    cv2.imshow(f'{arr.shape}',arr)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

In [3]:
# 加载图像数据并进行简单展示
image1 = cv2.imread('./01.png',cv2.IMREAD_ANYCOLOR)
myimshow(image1)

#### 一.方法1
* 为了避免在二值化的时候忽略部分低亮度点，将像素值低于120的像素点周围的像素点置为0
* 进行二值化操作将目标区域转换为高亮区域
* 膨胀目标区域便于后面提取轮廓

In [ ]:
# 转换为灰度图
image2 = cv2.cvtColor(image1,cv2.COLOR_BGR2GRAY)
myimshow(image2)
# 将低亮度周围区域也置为0
for i in range(image2.shape[0]):
    for j in range(image2.shape[1]):
        if image2[i,j]<120:
            image2[i-1:i+1,j-1:j+1] = 0

# 二值化操作
t,thresMat = cv2.threshold(image2, 120, 255, cv2.THRESH_BINARY_INV)
# 3.膨胀扩大目标区域
# 类似卷积核
kernel = np.ones(shape=(3,3),dtype=np.uint8)
thresMat1 = cv2.dilate(thresMat,kernel,iterations=2)
myimshow(thresMat)

#### 二.方法1
* 首先对灰度图进行核大小为(3*3)的均值滤波，能够凸显边缘区域
* 不固定阈值范围，使用自适应阈值选择
* 使用形态学操作膨胀，将目标区域扩大

In [11]:
# 转换为灰度图
image2 = cv2.cvtColor(image1,cv2.COLOR_BGR2GRAY)
myimshow(image2)
# 1.先对图像做一个3*3的均值滤波
filter = cv2.blur(image2,(3,3))
# 2.通过THRESH_OTSU 自动寻找合适的阈值，适合双峰，需要把阈值参数设置为0
t,thresMat = cv2.threshold(image2, 0, 255, cv2.THRESH_BINARY_INV | cv2.THRESH_OTSU)
# 3.膨胀扩大目标区域
# 卷积核选择一个低亮度毛孔区域，使用labelIMG标注工具获取坐标信息,进而获取ROI区域
"""
<bndbox>
			<xmin>258</xmin>
			<ymin>224</ymin>
			<xmax>262</xmax>
			<ymax>229</ymax>
		</bndbox>
"""
kernel_1 = image2[258:263,224:230]
# kernel = np.ones(shape=(3,3),dtype=np.uint8)
thresMat1 = cv2.dilate(thresMat,kernel_1,iterations=2)
myimshow(thresMat)

In [12]:
# 卷积核大小为4*5，具体像素值如下
kernel_1

array([[135, 134, 135, 136, 139, 140],
       [133, 132, 134, 135, 136, 137],
       [132, 132, 134, 134, 135, 137],
       [128, 130, 132, 133, 135, 137],
       [126, 124, 126, 128, 131, 133]], dtype=uint8)

In [13]:
# 获取毛孔轮廓(mode=只检测外轮廓，method=只保留终点坐标)
contours, hierarchy = cv2.findContours(thresMat, mode=cv2.RETR_EXTERNAL, method=cv2.CHAIN_APPROX_SIMPLE)
# 框出选择出来的区域
image3 = cv2.drawContours(image2.copy(), contours, contourIdx=-1, color=(0, 0, 255), thickness=2)
# 进行图像拼接
total_2 = np.concatenate((image2,thresMat,image3),axis=-1)
myimshow(total_2)

In [14]:
# 获取模板所有轮廓的外接矩形
areas = []
for contour in contours:
    x, y, w, h = cv2.boundingRect(contour)
    ar = w / float(h)
    # 对不符合条件的轮廓外接矩形进行筛选
    
    if ar < 2 and (0 < w < 20) and (0 < h < 20):
        areas.append((x, y, w, h))


In [15]:
image1_copy  = image1.copy()
for idx,(x,y,w,h)in enumerate(areas):
    cv2.rectangle(image1_copy, (x-2, y-2), (x + w+2, y + h+2), color=(0, 0, 255), thickness=1)
    # cv2.putText(image1_copy, 'pore', (x, y - 15), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 0, 255), 2)
myimshow(image1_copy)

In [16]:
cv2.imwrite('./pore3.jpg',image1_copy)

True

In [9]:
img0 = cv2.imread('./pore0.jpg')
img1 = cv2.imread('./pore.jpg')
img2 = cv2.imread('./pore1.jpg')
img3 = cv2.imread('./pore2.jpg')
img_total = np.concatenate((img0,img1,img2,img3),axis=1)
myimshow(img_total)
cv2.imwrite('./compare.jpg',img_total)

True